# Initialization

In [ ]:
%matplotlib inline

In [ ]:
# data access
import io
import sqlalchemy as sa

# data handling
import json

# internet
import requests

# data analysis
import numpy as np
import pandas as pd

#
from sklearn.impute import SimpleImputer
from sklearn import decomposition

import scipy
from scipy import stats
import statsmodels.api as sm
#import scikit-learn as sk

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# system packages
from imp import reload
import time

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.style.use('seaborn')

In [ ]:
# my own libs
from libs.gov_eneff import rt

rt.init('config/config.json')
logger = rt.logger

In [ ]:
from libs.gov_eneff import db
reload(db)

In [ ]:
from libs.gov_eneff import preprocessing
reload(preprocessing)

In [ ]:
#pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 50)

In [ ]:
LANG='en'

# Load Data

In [ ]:
db = db.DB()
db.load(source='local')

In [ ]:
df_org      = db.df_org.copy()
df_declr    = db.df_declr.copy()
df_bld      = db.df_bld.copy()

In [ ]:
df_bld = df_bld.merge(df_org['label'], left_on='id_org', right_index=True, how='left')

# Attributes

List of attributes to work with.

## Adding New Attributes

In [ ]:
df_bld['meter_heat_inputs_exists'] = df_bld['meter_heat_inputs_commercial'] > 0

## Sets of Attributes

The most complete list

In [ ]:
attr_set_base = [
    'id_voc_building_function_type',
    'id_voc_building_type',
    'floor_area',
    'useful_area',
    'heated_area',
    'total_value',
    'floor_amount',
    #'launch_year',
    #'building_age',
    'id_voc_outer_walls_list',
    'front_insulated',
    'id_voc_windows_list',
    'id_voc_windows_glazing',
    'windows_glazing_coverage',
    # 'windows_other', # this is a textual field
    'id_voc_windows_wood',
    'single_door_amount',
    'double_door_amount',
    'entrance_amount',
    'vestibule_amount',
    'door_closer_amount',
    'air_curtain_amount',
    'air_curtain_manual_amount',
    'air_curtain_auto_amount',
    'roof_exists',
    'roof_frost',
    'roof_insulated',
    'roofing_metal_insulated',
    'roofing_soft_single_insulation',
    'attic_exists',
    'attic_insulated',
    'attic_pipe_insulated',
    'technical_floor_exists',
    'basement_exists',
    'basement_cold',
    'basement_damp',
    'basement_walls_freeze',
    'basement_glazing',
    'basement_pipe_insulated',
    'id_voc_boiler_control',
    # 'meter_heat_inputs_commercial',  # replaced by meter_heat_inputs_exists
    'id_voc_heat_piping',
    'cast_iron_heater_amount',
    'convectors_amount',
    'convectors_with_thermostatic_flow_control',
    'bimetal_heater_amount',
    'thermostatic_control_heater_amount',
    'individual_control_heater_amount',
    'additional_heater_amount',
    'other_heaters_amount',
    'central_ventilation_exists',
    'employees',
    'guests',
    'central_dispatching_exists',
    #'heat_points',
    # 'heat_payment',
    # 'main_okved_code',
    # 'main_okved_label',
    'top_org_is_municipality',
    # 'heat_consumption_per_heated_area',
    # 'main_okved_code_l1',
    'dd_okved_code_l1',
    'ya_climate_zone',
    'dd_climate_zone',
    'meter_heat_inputs_exists']

In [ ]:
attr_set1 = attr_set_base.copy()

# Imputation (of Missing Values)

In [ ]:
# keeping information about what values were missing

from sklearn.impute import MissingIndicator

indicator = MissingIndicator()
indicator.fit(df_bld)
bld_missing_values = indicator.transform(df_bld)

print(bld_missing_values.mean())

In [ ]:
attr_with_missing_values = round(100*df_bld[attr_set1].isna().mean().sort_values(ascending = False),1)
attr_with_missing_values[attr_with_missing_values > 0]

In [ ]:
df_bld = preprocessing.i_bld_impute(df_bld)

In [ ]:
df_bld.shape

In [ ]:
attr_with_missing_values = round(100*df_bld[attr_set1].isna().mean().sort_values(ascending = False),1)
attr_with_missing_values[attr_with_missing_values > 0]

# Types Normalization

In [ ]:
df_bld = preprocessing.t_bld_categorize(df_bld)

In [ ]:
#df_bld['dd_okved_code_l1'] = df_bld['dd_okved_code_l1'].astype('category')

# EDA (All Data)

## Heat Consumption but No Heat Payment

In [ ]:
(df_bld['heat_consumption'].notna() & df_bld['heat_payment'].isna()).mean()

## Distribution Over Years

In [ ]:
# xx-small, x-small, small, medium, large, x-large, xx-large, larger, smaller

SMALL_SIZE = 10
MEDIUM_SIZE = 12
LARGE_SIZE = 14

plt.rc('font',   size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes',   titlesize=LARGE_SIZE)      # fontsize of the axes title
plt.rc('axes',   labelsize=LARGE_SIZE)      # fontsize of the x and y labels
plt.rc('xtick',  labelsize=LARGE_SIZE)      # fontsize of the tick labels
plt.rc('ytick',  labelsize=LARGE_SIZE)      # fontsize of the tick labels
plt.rc('legend', fontsize=LARGE_SIZE)       # legend fontsize
plt.rc('figure', titlesize=LARGE_SIZE)      # fontsize of the figure title

As is

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld.groupby('year')['heat_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Потребление тепла, млн. Гкал')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('Heat consumption, mln. GCal')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_consumption_years_{LANG}.png', dpi=300)
fig.savefig(f'images/heating/bld_heat_consumption_years_{LANG}.pdf')

In [ ]:
print(round(df_bld.groupby('year')['heat_consumption'].sum()/1000000, 1))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld\
    .groupby('year')['heat_payment'].sum()/1000000

dft.plot(kind='bar', ax=ax)
if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Оплата тепла, млн. руб.')

plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_payment_years_{LANG}.png', dpi=300)

## Examples of Outliers

In [ ]:
df_bld['chk_floor_heated_area_diff'] = df_bld['floor_area'] - df_bld['heated_area']
df_bld['chk_floor_useful_area_diff'] = df_bld['floor_area'] - df_bld['useful_area']

In [ ]:
# Указанная общая площадь меньше отапливаемой

df_bld.query('chk_floor_heated_area_diff < 0')\
    [['year', 'id_org', 'floor_area', 'heated_area', 'useful_area', 'chk_floor_heated_area_diff', 'heat_consumption']]\
    .sort_values('chk_floor_heated_area_diff', ascending=True)\
    .merge(df_org[['label']], left_on='id_org', right_index=True).head(10)

In [ ]:
# Указанная общая площадь меньше полезной

df_bld.query('chk_floor_useful_area_diff < 0')\
    [['year', 'id_org', 'floor_area', 'heated_area', 'useful_area', 'chk_floor_useful_area_diff', 'heat_consumption']]\
    .sort_values('chk_floor_useful_area_diff', ascending=True)\
    .merge(df_org[['label']], left_on='id_org', right_index=True).head(10)

In [ ]:
# Общая площадь – нулевая, потребление тепла – есть.

df_bld.query('floor_area == 0')\
    [['year', 'id_org', 'floor_area', 'heated_area', 'useful_area', 'heat_consumption']]\
    .sort_values('heat_consumption', ascending=False)\
    .merge(df_org[['label']], left_on='id_org', right_index=True).head(10)

In [ ]:
# Отапиваемая площадь – нулевая, потребление тепла – есть.

df_bld.query('heated_area == 0')\
    [['year', 'id_org', 'floor_area', 'heated_area', 'useful_area', 'heat_consumption']]\
    .sort_values('heat_consumption', ascending=False)\
    .merge(df_org[['label']], left_on='id_org', right_index=True).head(10)

In [ ]:
# Ненормально высокое потребление (heat_consumption_per_heated_area)

df_bld[df_bld['heat_consumption_per_heated_area'] > df_bld['heat_consumption_per_heated_area'].quantile(0.99)]\
    [['heated_area', 'heat_consumption', 'heat_consumption_per_heated_area', 'label']]\
    .sort_values('heat_consumption_per_heated_area', ascending=False)

### Building with High Ratio of Min/Max Consumption over Years

In [ ]:
# number of years
years_count = len(df_bld['year'].drop_duplicates())

# let's try to find such buildings which have too wide range over the year, 
# let's say their difference from the median is more than 50% for, at least, one year
df_bld_ht_diff = df_bld.sort_values(['id_org_building', 'year'])\
    .groupby(['id_org_building'])['year', 'id_org_building', 'heat_consumption']\
    .filter(lambda x: len(x['heat_consumption']) == years_count and (x['heat_consumption'] > 0).all())\
    .groupby(['id_org_building'])['heat_consumption']\
    .apply(lambda x: x.max()/x.min())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax = sns.distplot(np.log10(df_bld_ht_diff))

if LANG =='ru':
    ax.set_title('Распределение отношения потребления тепла по годам: $r=\log_{10}{\\frac{x_{max}}{x_{min}}}$')
    ax.set_xlabel('Отношение')
elif LANG == 'en':
    ax.set_xlabel('r (ratio)')
    
ax.set_ylabel('$f(r)$')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_consumption_ratio_distribution_{LANG}.png', dpi=300)
fig.savefig(f'images/heating/bld_heat_consumption_ratio_distribution_{LANG}.pdf')

In [ ]:
df_bld_ht_diff.loc[lambda x: x >= 100].sort_values().head(5)

In [ ]:
df_bld.query('id_org_building == 38715')[['id_org', 'id_org_building', 'year', 'heat_consumption']].sort_values('year')\
    .merge(df_org['label'], left_on='id_org', right_index=True)

# Filtering Out

In [ ]:
bld_hc_filter1 = preprocessing.Filter()

bld_hc_filter1.add(name='bld_hc_exists', fn=preprocessing.f_bld_hc_exists)
bld_hc_filter1.add(name='bld_hc_reasonable_areas', fn=preprocessing.f_bld_hc_areas)
bld_hc_filter1.add(name='bld_hc_consistency', fn=preprocessing.f_bld_hc_consistency)
bld_hc_filter1.add(name='bld_hc_no_outliers', fn=preprocessing.f_bld_hc_no_outliers)
bld_hc_filter1.add(name='bld_hc_inputs_commercial', fn=preprocessing.f_bld_hc_inputs_commercial)
bld_hc_filter1.add(name='bld_hc_inputs_com_exists', fn=preprocessing.f_bld_hc_inputs_com_exists)
bld_hc_filter1.add(name='bld_hc_bld_employees', fn=preprocessing.f_bld_hc_employees)

print(bld_hc_filter1)

In [ ]:
df_bld_clean = bld_hc_filter1.transform(df_bld).copy()

In [ ]:
logger.info('The data left of the original after all the filters: {:.2f}%'.format(100*len(df_bld_clean)/len(df_bld), 2))

# EDA (Clean Data)

In [ ]:
df_bld_clean['year'].value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld_clean.groupby('year')['heat_consumption'].sum()/1000000

dft.plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Потребление тепла, млн. Гкал')
elif LANG == 'en':
    ax.set_xlabel('Year')
    ax.set_ylabel('Heat consumption, mln. GCal')
    
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_consumption_years_filtered_subset_{LANG}.png', dpi=300)
fig.savefig(f'images/heating/bld_heat_consumption_years_filtered_subset_{LANG}.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld_clean.groupby('year')['heat_payment'].sum()/1000000

dft.plot(kind='bar', ax=ax)

if LANG =='ru':
    ax.set_xlabel('Год')
    ax.set_ylabel('Оплата тепла, млн. руб.')

plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_payment_years_filtered_subset_{LANG}.png', dpi=300)
fig.savefig(f'images/heating/bld_heat_payment_years_filtered_subset_{LANG}.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

sns.distplot(df_bld_clean['heat_consumption_per_heated_area'])

if LANG =='ru':
    ax.set_xlabel('Распределение отношения потребления тепла к площади, $ГКал/m_2$')
elif LANG == 'en':
    ax.set_xlabel('Heat consumption per heated area per year, $GCal/m_2$')    

ax.set_ylabel('$f$')

fig.tight_layout()
fig.savefig(f'images/heating/bld_heat_consumption_per_heated_area_filtered_{LANG}.png', dpi=300)
fig.savefig(f'images/heating/bld_heat_consumption_per_heated_area_filtered_{LANG}.pdf')

## Distributions (Boxplots)

In [ ]:
years = (2017, 2018)
dft = df_bld_clean.query('year in @years').copy()
dft['year'] = dft['year'].cat.remove_unused_categories()

In [ ]:
# let's build series of boxplots and regression lines per each factor re the heat consumption

for col_name in (
        'id_voc_building_function_type',
        'id_voc_building_type',
        'floor_area',
        'useful_area',
        'heated_area',
        'total_value',
        'floor_amount',
        'launch_year',
        'building_age',
        'id_voc_outer_walls_list',
        'front_insulated',
        'id_voc_windows_list',
        'id_voc_windows_glazing',
        'windows_glazing_coverage',
        'windows_other',
        'id_voc_windows_wood',
        'single_door_amount',
        'double_door_amount',
        'entrance_amount',
        'vestibule_amount',
        'door_closer_amount',
        'air_curtain_amount',
        'air_curtain_manual_amount',
        'air_curtain_auto_amount',
        'roof_exists',
        'roof_frost',
        'roof_insulated',
        'roofing_metal_insulated',
        'roofing_soft_single_insulation',
        'attic_exists',
        'attic_insulated',
        'attic_pipe_insulated',
        'technical_floor_exists',
        'basement_exists',
        'basement_cold',
        'basement_damp',
        'basement_walls_freeze',
        'basement_glazing',
        'basement_pipe_insulated',
        'id_voc_boiler_control',
        'meter_heat_inputs_commercial',
        'id_voc_heat_piping',
        'cast_iron_heater_amount',
        'convectors_amount',
        'convectors_with_thermostatic_flow_control',
        'bimetal_heater_amount',
        'thermostatic_control_heater_amount',
        'individual_control_heater_amount',
        'additional_heater_amount',
        'other_heaters_amount',
        'central_ventilation_exists',
        'employees',
        'guests',
        'central_dispatching_exists',
        'heat_points',
        'heat_payment',
        'main_okved_code',
        'main_okved_label',
        'top_org_is_municipality',
        'heat_consumption_per_heated_area',
        'main_okved_code_l1',
        'dd_geo_lat',
        'dd_geo_lon',
        'dd_okved',
        'dd_okved_code_l1',
        'ya_climate_zone',
        'dd_climate_zone',
        'meter_heat_inputs_exists'
    ):
    
    if pd.api.types.is_categorical_dtype(dft[col_name].dtype) or \
       pd.api.types.is_object_dtype(dft[col_name].dtype):
        
        # getting the number of distinct values for a categorical/object variable
        num_x_values = len(dft[col_name].drop_duplicates())
        
    elif pd.api.types.is_bool_dtype(dft[col_name].dtype):
        
        num_x_values = 2
        
    else:
        
        num_x_values = None
        
    if num_x_values is None:
        fig_long_side = 6
    else:
        fig_long_side = max(4, int(num_x_values/1.5))

    logger.debug(f'Processing {col_name}  with {num_x_values} of unique values, the figure width is {fig_width}')
   
    # box plot for categorical x
    if num_x_values is not None:
        
        #fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(fig_long_side, 8))
        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, fig_long_side))
        ax = axs
        #sns.boxplot(x=col_name, y="heat_consumption", hue='year', data=dft, palette="Set3", ax=ax)
        sns.boxplot(y=col_name, x="heat_consumption", hue='year', data=dft, palette="Set3", ax=ax)
        
        # checking if need to rotate the x ticks labels
        max_tick_length = max([len(s.get_text()) for s in ax.get_xticklabels()])
        
        # clipping the labels if they are too lengthy
        
        ax.set_xticklabels([s.get_text()[:16] for s in ax.get_xticklabels()])
        
        if max_tick_length >= 4 and max_tick_length < 6:
            ax.tick_params(axis='x', labelrotation=45)
        elif max_tick_length >= 6:
            ax.tick_params(axis='x', labelrotation=-90)
            
    # lmplot for numeric x
    else:
        
        if len(dft.year.cat.categories) == 2:
            markers=['o', 'x']
        elif len(dft.year.cat.categories) == 3:
            markers=['o', 'x', '-']            
        else:
            raise ValueError('Not enough markers')
        
        g = sns.lmplot(x=col_name, y="heat_consumption", hue="year", data=dft, markers=markers,
                         scatter_kws={'alpha': 0.5, 's': 20}, x_jitter=.3, y_jitter=0.3)
        
        fig = g.fig
    
    fig.suptitle(f'{col_name}')
    
    #fig.canvas.draw()
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(f'images/heating/bld_heating_factor_{col_name}.png', dpi=150)
    fig.savefig(f'images/heating/bld_heating_factor_{col_name}.pdf')
    
    time.sleep(1)
    
    plt.close()

# Group Comparisions

## Existence of Heat Input

In [ ]:
bld_hc_filter2 = preprocessing.Filter()

bld_hc_filter2.add(name='bld_hc_exists', fn=preprocessing.f_bld_hc_exists)
bld_hc_filter2.add(name='bld_hc_reasonable_areas', fn=preprocessing.f_bld_hc_areas)
bld_hc_filter2.add(name='bld_hc_consistency', fn=preprocessing.f_bld_hc_consistency)
bld_hc_filter2.add(name='bld_hc_no_outliers', fn=preprocessing.f_bld_hc_no_outliers)
bld_hc_filter2.add(name='bld_hc_bld_employees', fn=preprocessing.f_bld_hc_employees)

print(bld_hc_filter2)

dft = bld_hc_filter2.transform(df_bld).copy()

logger.info('The data left of the original after all the filters: {:.2f}%'.format(100*len(dft)/len(df_bld), 2))

In [ ]:
x = dft[dft['meter_heat_inputs_exists']]['heat_consumption'].dropna()
y = dft[dft['meter_heat_inputs_exists'] == False]['heat_consumption']
scipy.stats.mannwhitneyu(x, y, alternative='two-sided')

## Front Insulated

In [ ]:
x = dft[dft['front_insulated'] == 0]['heat_consumption'].dropna()
y = dft[dft['front_insulated'] == 1]['heat_consumption'].dropna()
print(len(x))
print(len(y))
scipy.stats.mannwhitneyu(x, y, alternative='two-sided')

## Roof

In [ ]:
dft['roof_exists'].value_counts()

In [ ]:
x = dft[dft['roof_exists'] == 0]['heat_consumption'].dropna()
y = dft[dft['roof_exists'] == 1]['heat_consumption'].dropna()
print(len(x))
print(len(y))
scipy.stats.mannwhitneyu(x, y, alternative='less')

## Attic

In [ ]:
dft['attic_exists'].value_counts()

In [ ]:
x = dft[dft['attic_exists'] == 0]['heat_consumption'].dropna()
y = dft[dft['attic_exists'] == 1]['heat_consumption'].dropna()
print(len(x))
print(len(y))
scipy.stats.mannwhitneyu(x, y, alternative='greater')

# Subset for Modelling

In [ ]:
years = (2014, 2015, 2016, 2017)
df_bld_model = df_bld_clean.query('year in @years').copy()

In [ ]:
df_bld_model['year'] = df_bld_model['year'].cat.remove_unused_categories()

# Useful Transformations

## Functional Usage

In [ ]:
## OKVED2

In [ ]:
import re

In [ ]:
def get_func_type(x):

    okved_code = x['dd_okved']
    
    if okved_code.startswith('85.11'):
        func_type = 'education_elementary'
    elif okved_code.startswith('85.1'):
        func_type = 'education_school'
    elif okved_code.startswith('85.41'):
        func_type = 'education_additional'
    elif okved_code.startswith('86.10'):
        func_type = 'health_hospitals'
    elif okved_code.startswith('86'):
        func_type = 'health_others'
    else:
        func_type = 'others'
        
    return(func_type)

In [ ]:
df_bld_model['func_type'] = df_bld_model.apply(lambda x: get_func_type(x), axis=1).astype('category')
df_bld_model.groupby('func_type')['func_type'].count().sort_values(ascending=False)

## Temperatures

In [ ]:
avg_temp_dict = {
    2014: {1: 6.0, 2: 6.9, 3: 6.2},
    2015: {1: 6.6, 2: 7.4, 3: 6.9},
    2016: {1: 5.8, 2: 6.7, 3: 6.0},
    2017: {1: 5.4, 2: 6.3, 3: 5.9},
    2018: {1: np.nan, 2: np.nan, 3: np.nan},
}

df_bld_model['avg_temp'] = df_bld_model.apply(lambda x: avg_temp_dict[x['year']][x['ya_climate_zone']], axis=1)

## Normalization

In [ ]:
df_bld_model['employees_t1']   = np.log10(df_bld_clean['employees'])
df_bld_model['total_value_t1'] = np.log10(df_bld_clean['total_value'])

## Box-Cox Transformation

In [ ]:
from scipy.special import boxcox, inv_boxcox

# normalizing the dependent variable

y = df_bld_model['heat_consumption'].to_numpy()

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,8))

g = sns.distplot(y, ax=axs[0][0])
t = axs[0][0].set_title('KDE of $y$')

r = stats.probplot(y, plot=axs[0][1], )

# Box-Cox transformation

# be aware of the issue with automatic findind of lambda: https://github.com/scipy/scipy/issues/6873
# this is the reason the lambda coefficient is calculated separetely with boxcox_normmax
bc_lmax = stats.boxcox_normmax(y, brack=(-2.1, 2.1),  method='mle')
# sadly, boxcox_normplot does not produce the graph, mostly probable due to the issue above
prob = stats.boxcox_normplot(y, -2, 2, plot=axs[0][2])
axs[0][2].axvline(bc_lmax, color='r')
axs[0][2].annotate('$\lambda=%.2f$' % bc_lmax, (0, 0.9))
axs[0][2].plot(prob[0], prob[1])

y_bc = stats.boxcox(x=y, lmbda=bc_lmax)

# manually going over a range lambda's
# TBC

g = sns.distplot(y_bc, ax=axs[1][0])
t = axs[1][0].set_title('KDE of $boxcox(y, \lambda)$')

# print('lambda: %f, CI: [%f,%f]' % (lmax, ci[0], ci[1]))
r = stats.probplot(y_bc, plot=axs[1][1])

axs[1][2].annotate('Empty', (0.1, 0.5))

fig.tight_layout()
fig.savefig('images/heating/regr_ht_bc_plots.png', dpi=300)
fig.savefig('images/heating/regr_ht_bc_plots.pdf')

In [ ]:
df_bld_model['heat_consumption_bc'] = y_bc

# Clusterization

# Reduction

## Area Attributes

In [ ]:
def get_area_attrs(df):
    
    return(df[['floor_area', 'heated_area']])

In [ ]:
np.random.seed(42)

pca_area = decomposition.PCA(n_components=2)
pca_area.fit(get_area_attrs(df_bld_model))

print('Percentage of variance explained by each of the components: %s' % pca_area.explained_variance_ratio_)
print('Estimated number of components: %d' % pca_area.n_components_)

In [ ]:
def transform_area(pca_area, df):
    
    df['area_pc1'] = pca_area.transform(get_area_attrs(df)).T[:1][0]
    
    return(df)

## Functionality

In [ ]:
def get_func_attrs(df):
    
    okved_dummies = pd.get_dummies(df['dd_okved_code_l1'], prefix='okved_code')
    bld_type_dummies = pd.get_dummies(df['id_voc_building_type'], prefix='bld_type')
    bld_fn_type_dummies = pd.get_dummies(df['id_voc_building_function_type'], prefix='bld_fn_type')
    
    dfr = pd.concat([okved_dummies, bld_type_dummies, bld_fn_type_dummies], axis=1)
    
    return(dfr)

In [ ]:
from sklearn import decomposition

np.random.seed(42)

n_components = 10
pca_func = decomposition.PCA(n_components=n_components)
pca_func.fit(get_func_attrs(df_bld_model))

print('Percentage of variance explained by each of the components: %s' % pca_func.explained_variance_ratio_)
print('Estimated number of components: %d' % pca_func.n_components_)

fig, ax = plt.subplots(figsize = (6, 4), ncols=1)

explained_variance_cumsum = np.cumsum(pca_func.explained_variance_ratio_)

plt.plot(range(1,len(pca_func.explained_variance_ratio_)+1), 
         explained_variance_cumsum, color='g', lw=4, zorder=1)
ax.set_ylim(0, 1.01)
ax.set_xticks(np.arange(1, n_components+1))
ax.set_yticks(np.arange(0.50, 1.00, 0.05))
ax.set_xlabel('Number of components')
ax.set_ylabel('Total explained variance')
ax.hlines(0.9, 1, n_components+1, color='r')
ax.vlines(7, 0, 1, color='r')

ax.annotate('{:.2f}'.format(explained_variance_cumsum[6]), xy=(7, explained_variance_cumsum[6]), 
            xytext=(8, explained_variance_cumsum[6]-0.05),
            arrowprops=dict(arrowstyle='->', color='black'))

fig.tight_layout()
fig.savefig('images/heating/pca_func.png', dpi=300)
fig.savefig('images/heating/pca_func.pdf')

In [ ]:
def transform_func(pca_func, df):
    
    df['func_pc1'], df['func_pc2'], df['func_pc3'], df['func_pc4'], \
    df['func_pc5'], df['func_pc6'], df['func_pc7'] = \
        pca_func.transform(get_func_attrs(df)).T[:7]
    
    return(df)

## Building Construction

In [ ]:
def get_cons_attrs(df):

   
    dfr = pd.concat([pd.get_dummies(df['id_voc_outer_walls_list'], prefix='outer_walls'), 
#                      pd.get_dummies(df['front_insulated'], prefix='front_insulated'),
                     pd.get_dummies(df['id_voc_windows_list'], prefix='windows_list'), 
#                      pd.get_dummies(df['roof_exists']),
#                      pd.get_dummies(df['roof_frost']),
#                      pd.get_dummies(df['roof_insulated']),
#                      pd.get_dummies(df['roofing_metal_insulated']),
#                      pd.get_dummies(df['roofing_soft_single_insulation']),
#                      pd.get_dummies(df['attic_exists']),
#                      pd.get_dummies(df['attic_insulated']),
#                      pd.get_dummies(df['attic_pipe_insulated']),
#                      pd.get_dummies(df['technical_floor_exists']),
#                      pd.get_dummies(df['basement_exists']),
#                      pd.get_dummies(df['basement_cold']),
#                      pd.get_dummies(df['basement_damp']),
#                      pd.get_dummies(df['basement_walls_freeze']),
#                      pd.get_dummies(df['basement_glazing']),
#                      pd.get_dummies(df['basement_pipe_insulated'])
                    ],    
                    axis=1)

    return(dfr)

In [ ]:
np.random.seed(42)

n_components = 10
pca_cons = decomposition.PCA(n_components=n_components)
pca_cons.fit(get_cons_attrs(df_bld_model))

print('Percentage of variance explained by each of the components: %s' % pca_cons.explained_variance_ratio_)
print('Estimated number of components: %d' % pca_cons.n_components_)

fig, ax = plt.subplots(figsize = (10, 6), ncols=1)

plt.plot(range(1,len(pca_cons.explained_variance_ratio_)+1), 
         np.cumsum(pca_cons.explained_variance_ratio_), color='g', lw=4, zorder=1)
ax.set_ylim(0, 1.01)
ax.set_yticks(np.arange(0.50, 1.00, 0.05))
ax.set_xlabel('Number of components')
ax.set_ylabel('Total explained variance')
ax.hlines(0.9, 1, n_components+1, color='r')
ax.vlines(n_components, 0, 1, color='r')

fig.tight_layout()

In [ ]:
def transform_cons(pca_cons, df):
    
    df['cons_pc1'], df['cons_pc2'], df['cons_pc3'] = \
        pca_cons.transform(get_cons_attrs(df)).T[:3]
    
    return(df)

## Transformations Using Above

In [ ]:
df_bld_clean = transform_area(pca_area, df_bld_clean)
df_bld_clean = transform_func(pca_func, df_bld_clean)

In [ ]:
df_bld_model = transform_area(pca_area, df_bld_model)
df_bld_model = transform_func(pca_func, df_bld_model)
df_bld_model = transform_cons(pca_cons, df_bld_model)

## Set of Attrutes

In [ ]:
attr_set1 = attr_set_base.copy()

attr_set1.append('employees_t1')
attr_set1.remove('employees')

attr_set1.remove('total_value')
attr_set1.append('total_value_t1')

attr_set1.append('avg_temp')

attr_set1.append('area_pc1')
attr_set1.remove('floor_area')
attr_set1.remove('heated_area')
attr_set1.remove('useful_area')

attr_set1.remove('dd_okved_code_l1')
attr_set1.remove('id_voc_building_type')
attr_set1.remove('id_voc_building_function_type')
for i in range(1, 8):
    attr_set1.append(f'func_pc{i}')
    
attr_set1.append('func_type')

attr_set1.remove('id_voc_outer_walls_list')
for i in range(1, 4):
   attr_set1.append(f'cons_pc{i}')

# Correlations

In [ ]:
# Compute the correlation matrix
corr = df_bld_model[['heat_consumption', 'heat_consumption_bc'] + attr_set1].corr(method='spearman')

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.set(style="white")

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 12))

# green diverging colormap
# cmap = sns.light_palette("green", as_cmap=True)

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
g = sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

fig = g.figure
ax = fig.axes[0]

ax.tick_params(labelsize=10)

fig.tight_layout()

fig.savefig('images/heating/corr_heatmap.png', dpi=300)
fig.savefig('images/heating/corr_heatmap.pdf')

In [ ]:
sns.set()

## Pairplots

Let's build the pairplot for the most strong correlations

In [ ]:
dft = df_bld_model[['heat_consumption_bc', 'total_value_t1', 'area_pc1', 'func_pc1',
                    'func_pc2', 'employees_t1', 'avg_temp', 'func_type']]

ax = sns.pairplot(dft, kind='reg', diag_kind='kde', markers='+')

fig = ax.fig
#fig.set_size_inches(8, 8)

fig.tight_layout()
fig.savefig('images/heating/regr_ht_correlogram.png', dpi=300)
fig.savefig('images/heating/regr_ht_correlogram.pdf')

# Linear Regression (Statsmodels)

In [ ]:
years = [2014, 2015, 2016, 2017]

In [ ]:
formula = 'heat_consumption_bc ~ '

aic = 100000

sm_family = sm.families.Gaussian(sm.families.links.identity)

print('No  {:42s} {:6s}    {:8s} {:10}'.format('Variable', 'pvalue', 'AIC', 'Included'))
print('----------------------------------------------------------------------------')

i = 0

for attr in attr_set1:
    
    if formula == 'heat_consumption_bc ~ ':        
        formula_tmp = formula + attr
    else:
        formula_tmp = formula + ' + ' + attr
    
    glm_fitter = sm.formula.glm(formula_tmp,
                                data=df_bld_model.query('year in @years'),
                                family=sm_family,
                                #family=sm.families.Gamma()
                                )

    glm_mod = glm_fitter.fit()
    
    pvalue = glm_mod.pvalues.filter(like=attr, axis='index').median()
    aic_ratio = (aic-glm_mod.aic)/aic
    
    print(f'{i:2d}  {attr:42} {pvalue:1.2f}    {glm_mod.aic:6.1f}       ', end='')
    
    if pvalue <= 0.2 and aic_ratio >= 0.01:        
        formula = formula_tmp
        aic = glm_mod.aic
        print('Yes')
    else:
        print('No')
        
    i = i + 1
    
print('')
print(formula)

In [ ]:
len(attr_set1)

In [ ]:
# formula = 'heat_consumption_bc ~ +floor_amount'\
#                 '+single_door_amount+employees_t1+total_value_t1+area_pc1+func_type'\
#                 '+cast_iron_heater_amount'\
#                 '+ya_climate_zone+entrance_amount*employees_t1'\
#                 '+front_insulated+top_org_is_municipality+cons_pc1'

formula = "heat_consumption_bc ~ entrance_amount + employees_t1 + total_value_t1 + area_pc1 + func_pc1"

glm_fitter = sm.formula.glm(formula,
                            data=df_bld_model,
                            family=sm_family)

glm_mod = glm_fitter.fit()

glm_mod.summary()

In [ ]:
def inv_transform(y):
    
    return(inv_boxcox(y, bc_lmax))
    #return(y)

In [ ]:
### see the discussion https://stats.stackexchange.com/questions/356053/the-identity-link-function-does-not-respect-the-domain-of-the-gamma-family
# re the warning "The identity link function does not respect the domain of the Gamma family."

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15,10))

# fitted vs. observed
y = glm_mod.model.data.orig_endog.iloc[:, 0].to_numpy()
yhat = glm_mod.mu

from statsmodels.graphics.api import abline_plot

ax = axs[0][0]
ax.scatter(yhat, y, marker='+', alpha=0.5)
line_fit = sm.OLS(y, sm.add_constant(yhat, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)
ax.set_xlim(5, 25)
ax.set_ylim(5, 25)

ax.set_title('Fitted vs. Observed')
if LANG == 'ru':
    ax.set_xlabel('Моделированные')
    ax.set_ylabel('Декларированные')
elif LANG == 'en':
    ax.set_xlabel('Fitted')
    ax.set_ylabel('Observed')

# CI
#pred = mod1.get_prediction(df_bld_regr)
#pred_summary = pred.summary_frame(alpha=0.05)
#ax.scatter(pred_summary['mean_ci_lower'], y, color = 'red')

# Residual Dependence Plot
ax = axs[0][1]
ax.scatter(yhat, glm_mod.resid_pearson, marker='+', alpha=0.5)
ax.hlines(0, 0, max(np.max(yhat), 25))
ax.set_title('Residual Dependence Plot')
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')
ax.set_xlim(5, 25)
ax.set_ylim(-15, 15)

# Histogram of standardized deviance residuals:
resid = glm_mod.resid_deviance.copy()
resid_std = stats.zscore(resid)

ax = axs[0][2]
p = sns.kdeplot(resid_std, ax=ax)
#ax.hist(resid_std, bins=25)
ax.set_title('KDE of standardized deviance residuals')

##############################################################
# Real Values

y_real = inv_transform(y)
yhat_real = inv_transform(yhat)

# Fitted vs. Observed (Real Values)
ax = axs[1][0]
ax.scatter(yhat_real, y_real, marker='+', alpha=0.5)
line_fit = sm.OLS(yhat_real, sm.add_constant(y_real, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)
ax.set_xlim(0, 1200)
ax.set_ylim(0, 1200)

ax.set_title('Fitted vs. Observed')
if LANG == 'ru':
    ax.set_xlabel('Моделированные')
    ax.set_ylabel('Декларированные')
elif LANG == 'en':
    ax.set_xlabel('Fitted')
    ax.set_ylabel('Observed')
    
# Residual Dependence Plot

resid_pearson_real =  inv_transform(glm_mod.resid_pearson)

ax = axs[1][1]
ax.scatter(yhat_real, resid_pearson_real, marker='+', alpha=0.5)
ax.hlines(0, 0, max(np.max(yhat_real), 1200))
ax.set_title('Residual Dependence Plot')
ax.set_ylabel('Pearson Residuals')
ax.set_xlabel('Fitted values')
ax.set_xlim(0, 1200)
#ax.set_ylim(-20, 50)

# Histogram of standardized deviance residuals:
resid_real = inv_transform(glm_mod.resid_deviance).dropna()
resid_real_std = stats.zscore(resid_real)

ax = axs[1][2]
p = sns.kdeplot(resid_real_std, ax=ax)
#ax.hist(resid_std, bins=25)
ax.set_title('KDE of standardized deviance residuals')
    
fig.tight_layout()
fig.savefig(f'images/heating/regr_ht_glm_diag_{LANG}.png', dpi=300)

In [ ]:
(resid_pearson_real < 0).mean()

In [ ]:
(glm_mod.resid_pearson < 0).mean()

## Evaluation

In [ ]:
print(np.sum(y), np.sum(yhat))
print(100*np.sum((np.abs(y - yhat)/y))/len(y))

In [ ]:
print(np.sum(y_real), np.sum(yhat_real))

## Predictions

In [ ]:
pred = glm_mod.get_prediction(df_bld_model)
pred_summary = pred.summary_frame(alpha=0.05)

df_bld_model['heat_consumption_pred']          = inv_transform(pred_summary['mean'].to_list())
df_bld_model['heat_consumption_pred_ci_lower'] = inv_transform(pred_summary['mean_ci_lower'].to_list())
df_bld_model['heat_consumption_pred_ci_upper'] = inv_transform(pred_summary['mean_ci_upper'].to_list())

In [ ]:
y = df_bld_model['heat_consumption']
yhat = df_bld_model['heat_consumption_pred']

mape = 100*np.sum((np.abs(y-yhat)/y))/len(y)
print(mape)

In [ ]:
pd.options.display.float_format = '{:,.1f}'.format

df_bld_model[['id_org', 'id_org_building', 'year', 'heat_consumption', 'heat_consumption_pred', 
              'heat_consumption_pred_ci_lower', 'heat_consumption_pred_ci_upper']]\
    .sort_values(['id_org', 'id_org_building', 'year']).sample(10)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld_model.groupby('year')['heat_consumption', 'heat_consumption_pred', 
                                   'heat_consumption_pred_ci_lower', 'heat_consumption_pred_ci_upper']\
    .agg({'heat_consumption': sum, 'heat_consumption_pred': sum, 
          'heat_consumption_pred_ci_lower': sum,
          'heat_consumption_pred_ci_upper': sum}) / 1000000

dft[['heat_consumption', 'heat_consumption_pred']].plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Потребление тепла, млн. Гкал')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')
if LANG == 'ru':
    ax.legend(['Факт', 'Прогноз'])
elif LANG == 'en':
    ax.legend(['Observed', 'Predicted'], loc='lower right', frameon=True)
    
for i, year in enumerate(dft.index.drop_duplicates().sort_values().to_numpy()):
    ax.vlines(i+0.12, 
              dft.iloc[i]['heat_consumption_pred_ci_lower'], 
              dft.iloc[i]['heat_consumption_pred_ci_upper'], color='red', linewidth=4)

fig.tight_layout()
fig.savefig('images/heating/bld_heat_consumption_years_validated_subset_pred.png', dpi=300)
fig.savefig('images/heating/bld_heat_consumption_years_validated_subset_pred.pdf')

In [ ]:
pred = glm_mod.get_prediction(df_bld_clean)
pred_summary = pred.summary_frame(alpha=0.05)

df_bld_clean['heat_consumption_pred']          = inv_transform(pred_summary['mean'].to_list())
df_bld_clean['heat_consumption_pred_ci_lower'] = inv_transform(pred_summary['mean_ci_lower'].to_list())
df_bld_clean['heat_consumption_pred_ci_upper'] = inv_transform(pred_summary['mean_ci_upper'].to_list())

y = df_bld_clean['heat_consumption']
yhat = df_bld_clean['heat_consumption_pred']

print(sum(y), sum(yhat))
mape = 100*np.sum((np.abs(y-yhat)/y))/len(y)
print(mape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld_clean.groupby('year')['heat_consumption', 'heat_consumption_pred', 
                                   'heat_consumption_pred_ci_lower', 'heat_consumption_pred_ci_upper']\
    .agg({'heat_consumption': sum, 'heat_consumption_pred': sum, 
          'heat_consumption_pred_ci_lower': sum,
          'heat_consumption_pred_ci_upper': sum}) / 1000000

dft[['heat_consumption', 'heat_consumption_pred']].plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Потребление тепла, млн. Гкал')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')
if LANG == 'ru':
    ax.legend(['Факт', 'Прогноз'])
elif LANG == 'en':
    ax.legend(['Observed', 'Predicted (with CI)'], loc='lower right', frameon=True)
    
for i, year in enumerate(dft.index.drop_duplicates().sort_values().to_numpy()):
    ax.vlines(i+0.12, 
              dft.iloc[i]['heat_consumption_pred_ci_lower'], 
              dft.iloc[i]['heat_consumption_pred_ci_upper'], color='red', linewidth=3)    

fig.tight_layout()
fig.savefig('images/heating/bld_heat_consumption_years_pred.png', dpi=300)
fig.savefig('images/heating/bld_heat_consumption_years_pred.pdf')

# Linear Regression (LASSO)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_bld_model[attr_set3], 
                                                    df_bld_model['heat_consumption'].to_numpy(),
                                                    test_size=0.3, random_state=42)

In [ ]:
min_max_scaler = MinMaxScaler()

X_train_sc = min_max_scaler.fit_transform(X_train)
y_train_sc = min_max_scaler.fit_transform(y_train.reshape(-1, 1))

X_test_sc = min_max_scaler.fit_transform(X_test)
y_test_sc = min_max_scaler.fit_transform(y_test.reshape(-1, 1))

In [ ]:
# We use the base estimator LassoCV since the L1 norm promotes sparsity of features.
clf = LassoCV(cv=5)

In [ ]:
# Set a minimum threshold of 0.25
sfm = SelectFromModel(clf, threshold=0)
sfm.fit(X_train_sc, y_train_sc)

In [ ]:
# Reset the threshold till the number of features equals two.
# Note that the attribute can be set directly instead of repeatedly

while True:
    
    X_transform = sfm.transform(X_train_sc)
    n_features = X_transform.shape[1]
    print(f'{n_features}, {sfm.threshold}')
    
    if n_features <= 10:
        print()
        break
    
    sfm.threshold += 0.001

In [ ]:
# indexes
features_selected = sfm.get_support(indices=True)
print(features_selected)

In [ ]:
X_train.iloc[:, features_selected]

In [ ]:
lasso = Lasso()
lasso.fit(X_train.iloc[:, features_selected], y_train)

In [ ]:
train_score = lasso.score(X_train.iloc[:, features_selected], y_train)
test_score  = lasso.score(X_test.iloc[:, features_selected], y_test)

In [ ]:
coeff_used = np.sum(lasso.coef_ != 0)

print(f'Training score: {train_score}')
print(f'Test score: {test_score}')
print(f'Number of features used: {coeff_used}')

# kNN

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

In [ ]:
from sklearn import neighbors

np.random.seed(0)

# #############################################################################
# Fit regression model
n_neighbors = 5

knn = neighbors.KNeighborsRegressor(n_neighbors, weights='uniform')

In [ ]:
y_ = knn.fit(X_train, y_train).predict(X_train)

In [ ]:
sns.scatterplot(y_, y_train, )

In [ ]:
y_ = knn.fit(X_train, y_train).predict(X_train)
g = sns.jointplot(y_, y_train, kind="reg")

In [ ]:
y_ = knn.fit(X_train, y_train).predict(X_test)
g = sns.jointplot(y_, y_test, kind="reg")

# Random Forest

# SVR

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

Generating CI 
- with the bootstrapping technique: https://stats.stackexchange.com/questions/183230/bootstrapping-confidence-interval-from-a-regression-prediction/218988#218988

In [ ]:
attr_set2 = [
    'total_value',
    'floor_amount',
    'front_insulated',
    'windows_glazing_coverage',
    'single_door_amount',
    'double_door_amount',
    'entrance_amount',
    'door_closer_amount',
    'air_curtain_amount',
    'central_ventilation_exists',
    'central_dispatching_exists',
    'top_org_is_municipality',
    'ya_climate_zone',
    'meter_heat_inputs_exists',
    'employees',
    'area_pc1',
    'func_pc1']

In [ ]:
attr_set3 = [
    'total_value',
    'entrance_amount',
    'employees',
    'area_pc1',
    'func_pc1']

## Encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
years = [2014, 2015, 2016, 2017]
srv_attr_set = attr_set3

In [ ]:
cat_attrs = df_bld_model.query('year in @years')[srv_attr_set].dtypes == 'category'
cat_attrs_idx = [i for i, x in enumerate(cat_attrs.to_list()) if x]
cat_attrs_idx_no = [i for i, x in enumerate(cat_attrs.to_list()) if not x]
print(cat_attrs_idx)

In [ ]:
ct = ColumnTransformer([('passthrough', 'passthrough', cat_attrs_idx_no),
                        ('one-hot', OneHotEncoder(), cat_attrs_idx)])

X = ct.fit_transform(df_bld_model.query('year in @years')[srv_attr_set])
X.shape

In [ ]:
y = df_bld_model['heat_consumption']

## Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()  # Default behavior is to scale to [0,1]
X = scaler.fit_transform(X)

## Splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = X, X, y, y

## Model

In [ ]:
from sklearn import model_selection
from sklearn import svm

In [ ]:
# from sklearn.pipeline import make_pipeline

# svr_pipeline = make_pipeline(
#     #MinMaxScaler(),
#     svm.SVR(kernel='rbf', epsilon=0.01, C=800, gamma = 'scale'),
# )
# svr_pipeline.fit(X_train, y_train)
# yhat = svr_pipeline.predict(X_train)

# print(sum(y_train), sum(yhat))
# mape = 100*np.sum((np.abs(y_train-yhat)/y_train))/len(y_train)
# print(mape)

In [ ]:
param_grid = {'kernel':['rbf'], 'C':[1, 10, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

svr = svm.SVR(gamma="scale")
clf = model_selection.GridSearchCV(svr, param_grid, cv=5, n_jobs=8)
clf.fit(X_train, y_train.to_numpy())

In [ ]:
mean_train_score = clf.cv_results_['mean_train_score']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

ax.plot(param_grid['C'], mean_train_score, marker='d', zorder=1)
ax.scatter(x=param_grid['C'], y=mean_train_score, marker='d', color='red', zorder=2)

ax.set_xlabel('C')
ax.set_ylabel('$R^2$')
ax.set_ylim(0, 1.1)
ax.set_yticklabels(np.round(np.arange(0, 1.01, 0.2), 1))
    
fig.tight_layout()
fig.savefig('images/heating/bld_heat_consumption_svr_C.pdf')

In [ ]:
best_clf = clf.best_estimator_

In [ ]:
from sklearn.pipeline import make_pipeline

svr_pipeline = make_pipeline(
    #MinMaxScaler(),
    svm.SVR(kernel='rbf', C=100, gamma = 'scale'),
)
svr_pipeline.fit(X_train, y_train)
yhat = svr_pipeline.predict(X_train)

print(sum(y_train), sum(yhat))
mape = 100*np.sum((np.abs(y_train-yhat)/y_train))/len(y_train)
print(mape)

21.7

11.4

In [ ]:
df_bld_model['heat_consumption_pred_svr1']     = yhat
df_bld_model['heat_consumption_pred_ci_lower'] = np.nan
df_bld_model['heat_consumption_pred_ci_upper'] = np.nan

In [ ]:
pd.options.display.float_format = '{:,.1f}'.format

df_bld_model[['id_org', 'id_org_building', 'year', 'heat_consumption', 'heat_consumption_pred_svr', 
              'heat_consumption_pred_ci_lower', 'heat_consumption_pred_ci_upper']]\
    .sort_values(['id_org', 'id_org_building', 'year']).sample(10)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

dft = df_bld_model.groupby('year')['heat_consumption', 'heat_consumption_pred_svr', 
                                   'heat_consumption_pred_ci_lower', 'heat_consumption_pred_ci_upper']\
    .agg({'heat_consumption': sum, 'heat_consumption_pred_svr': sum, 
          'heat_consumption_pred_ci_lower': sum,
          'heat_consumption_pred_ci_upper': sum}) / 1000000

dft[['heat_consumption', 'heat_consumption_pred_svr']].plot(kind='bar', ax=ax)
ax.set_xlabel('Год')
ax.set_ylabel('Потребление тепла, млн. Гкал')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
#ax.ticklabel_format(style='plain')
#plt.ticklabel_format(style='plain', axis='y')
if LANG == 'ru':
    ax.legend(['Факт', 'Прогноз'])
elif LANG == 'en':
    ax.legend(['Observed', 'Predicted'], loc='lower right', frameon=True)
    
for i, year in enumerate(dft.index.drop_duplicates().sort_values().to_numpy()):
    ax.vlines(i+0.12, 
              dft.iloc[i]['heat_consumption_pred_ci_lower'], 
              dft.iloc[i]['heat_consumption_pred_ci_upper'], color='red', linewidth=4)

fig.tight_layout()
fig.savefig('images/heating/bld_heat_consumption_years_svr_pred.png', dpi=300)
fig.savefig('images/heating/bld_heat_consumption_years_svr_pred.pdf')

# Observed vs Predicted

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

ax = axs[0]
y = df_bld_model['heat_consumption']
yhat = df_bld_model['heat_consumption_pred']
ax.scatter(yhat, y, alpha=0.5, marker='+')
line_fit = sm.OLS(yhat_real, sm.add_constant(y, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)
ax.set_xlim(0, 1200)
ax.set_ylim(0, 1200)
ax.set_xlabel('Fitted')
ax.set_ylabel('Observed')
ax.set_title('GLM (limited set)')
ax.text(900, 1100, 'MAPE=24.7%')

ax = axs[1]
y = df_bld_model['heat_consumption']
yhat = df_bld_model['heat_consumption_pred_svr1']
ax.scatter(yhat, y, alpha=0.5, marker='+')
line_fit = sm.OLS(yhat_real, sm.add_constant(y, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)
ax.set_xlim(0, 1200)
ax.set_ylim(0, 1200)
ax.set_xlabel('Fitted')
ax.set_ylabel('Observed')
ax.set_title('SVR (limited set, C=200)')
ax.text(900, 1100, 'MAPE=21.7%')

ax = axs[2]
y = df_bld_model['heat_consumption']
yhat = df_bld_model['heat_consumption_pred_svr2']
ax.scatter(yhat, y, alpha=0.5, marker='+')
line_fit = sm.OLS(yhat_real, sm.add_constant(y, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)
ax.set_xlim(0, 1200)
ax.set_ylim(0, 1200)
ax.set_xlabel('Fitted')
ax.set_ylabel('Observed')
ax.set_title('SVR (full set, C=400)')
ax.text(900, 1100, 'MAPE=11.7%')

fig.tight_layout()
fig.savefig('images/heating/bld_heat_consumption_all_pred.png', dpi=300)